In [ ]:
import pandas as pd

dataset = pd.read_csv("spam_in_emails.csv")

In [ ]:
dataset.head()

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...


In [ ]:
# Tomamos la columna label y la colocamos en su propio vector numerico
import numpy as np

classes = dataset["label"]

In [ ]:
# La columna label que ya fue guardada en otro vector por lo que es eliminada
dataset = dataset.drop("label", axis=1)
dataset.head()

,text
0,ounce feather bowl hummingbird opec moment ala...
1,wulvob get your medircations online qnb ikud v...
2,computer connection from cnn com wednesday es...
3,university degree obtain a prosperous future m...
4,thanks for all your answers guys i know i shou...


In [ ]:
# Generamos conjuntos de entrenamiento y prueba y estratificamos
from sklearn.model_selection import train_test_split

# Escencialmente dividimos la base de datos en "dataset" la cual sería solo las características (x) y "classes" que sería solo la salida (Y)
xtrain, xtest, ytrain, ytest = train_test_split(dataset, classes, test_size=0.20, stratify=classes)

Desgraciadamente la estandarización y escalamiento de los datos no puede ser llevada acabo debido a que nuestras dos únicas variables en la base de datos son booleanos y cadenas de caracteres.

Lo que si debemos de hacer es transformar nuestras características a una forma númerica. Algunas opciones para hacer esto serían utilizar técnicas de codificación como la codificación one-hot para variables categóricas o la vectorización de texto para datos de texto.

Utilizaremos la técnica de vectorización de texto para convertir cada texto en un vector numérico que represente sus características.

Intente primero el siguiente código:

from sklearn.feature_extraction.text import CountVectorizer

% Inicializar el vectorizador

vectorizer = CountVectorizer()

% Ajustar el vectorizador a tus datos de entrenamiento y transformar los datos de entrenamiento

xtrain_vectorized = vectorizer.fit_transform(xtrain)

% Transformar los datos de prueba utilizando el mismo vectorizador

xtest_vectorized = vectorizer.transform(xtest)

Pero de esta manera vectorizo una única instancia de xtest o xtrain por lo que lo corregí con lo siguiente:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Concatenar xtrain y xtest antes de vectorizar
all_text = pd.concat([xtrain.squeeze(), xtest.squeeze()])

# Inicializar el vectorizador y ajustarlo a todos los datos
vectorizer = CountVectorizer()
x_all_vectorized = vectorizer.fit_transform(all_text)

# Dividir los datos vectorizados nuevamente en xtrain y xtest
xtrain_vectorized = x_all_vectorized[:len(xtrain)]
xtest_vectorized = x_all_vectorized[len(xtrain):]

# Ahora, xtest_vectorized debería tener la forma correcta
print("Shape of xtest_vectorized:", xtest_vectorized.shape)
print("Shape of ytest:", ytest.shape)

Shape of xtest_vectorized: (378, 34750)
Shape of ytest: (378,)


Comenzamos el entrenamiento

In [ ]:
from sklearn.svm import SVC

# Crear el clasificador SVM con un kernel lineal
svm_clf = SVC(kernel="linear")

# Entrenar el clasificador utilizando los datos de entrenamiento
svm_clf.fit(xtrain_vectorized, ytrain)

SVC(kernel='linear')

Ahora que está entrenado podemos realizar predicciones

In [ ]:
predictions = svm_clf.predict(xtest_vectorized)

Ahora que fueron realizadas las predicciones solo nos queda ver como nos fue

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

accuracy = accuracy_score(ytest, predictions)
print("Accuracy:", accuracy)
cm = confusion_matrix(ytest, predictions)
print("Confusion matrix: \n",cm)
# Probamos primero el f1 score con la clase 0 o "no spam"
f1_0 = f1_score(ytest, predictions, pos_label=0)
print(f1_0)
# Probamos ahora el f1 score con la clase 1 o "spam"
f1_1 = f1_score(ytest, predictions, pos_label=1)
print(f1_1)

Accuracy: 0.9365079365079365
Confusion matrix: 
 [[163  12]
 [ 12 191]]
0.9314285714285714
0.9408866995073891
